**First, do imports.**

In [1]:
import xarray as xr
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib.pyplot as plt

**Next, we want to load in one dataset and specify our variables.**

In [2]:
ds = xr.open_dataset('cm1_original/cm1out_20000101.000004.nc').isel(time=0)

u = ds['u']
v = ds['v']
w = ds['w']
dx = ds['xh']*units('m')
dy = ds['yh']*units('m')
dz = ds['zh']*units('m')

**Then, calculate relative vorticity using metpy.**

In [4]:
dvdx = mpcalc.first_derivative(v, delta=dx[1]-dx[0])
dudy = mpcalc.first_derivative(u, delta=dy[1]-dy[0])
z_vort = dvdx-dudy

**Horizontal advection term.**

In [5]:
dvortdx = mpcalc.first_derivative(z_vort, delta=dx[1]-dx[0])
dvortdy = mpcalc.first_derivative(z_vort, delta=dy[1]-dy[0])

udvortdx = u*dvortdx
vdvortdy = v*dvortdy

horiz_adv = -1*(udvortdx+vdvortdy)

**Vertical advection term.**

In [6]:
dvortdz = mpcalc.first_derivative(z_vort, delta=dz[1]-dz[0])

In [7]:
dvortdz

wdvortdz = w*dvortdz

vert_adv = -1*wdvortdz

**Stretching Term.**

In [20]:
dudx = mpcalc.first_derivative(u, delta=dx[1]-dx[0])
dvdy = mpcalc.first_derivative(v, delta=dy[1]-dy[0])

div = dudx+dvdy

def stretching_term(dudx, dvdy, z_vort):
    return(z_vort*(dudx+dvdy))
stretch_term = stretching_term(dudx,dvdy,z_vort)

In [22]:
stretch_term.isel(nk=0)

AttributeError: Neither Quantity object nor its magnitude ([[[-3.77404876e-02 -4.58110496e-02 -9.79291946e-02 ...  3.13750841e-02
    3.94491479e-02  4.49873023e-02]
  [-2.64699869e-02 -4.32312265e-02 -7.67382979e-02 ...  3.08423657e-02
    3.55080143e-02  4.89895307e-02]
  [-1.99246854e-02 -2.68708654e-02 -2.38234289e-02 ...  2.59417687e-02
    3.26443315e-02  5.09426706e-02]
  ...
  [-1.63004699e-03  1.67395410e-04 -2.77883606e-03 ... -1.64065454e-02
   -1.50811654e-02 -1.22408429e-02]
  [-1.94054726e-03  6.98049320e-04 -6.20496739e-03 ... -1.79297142e-02
   -1.23330988e-02 -7.55379861e-03]
  [-1.62910542e-03  8.18429166e-04 -7.15203863e-03 ... -8.72200914e-03
   -5.03418641e-03 -6.49844529e-03]]

 [[-1.68888585e-03 -4.24946705e-03 -9.10686236e-03 ...  7.09772855e-03
    9.56568774e-03  1.13598062e-02]
  [-2.90484773e-03 -4.07839101e-03 -7.84595590e-03 ...  6.64618937e-03
    9.32910480e-03  1.18747158e-02]
  [-6.29095500e-03 -7.48371007e-03 -6.64391881e-03 ...  6.70715841e-03
    8.

**Tilting Term.**

In [9]:
dwdx = mpcalc.first_derivative(w, delta=dx[1]-dx[0])
dvdz = mpcalc.first_derivative(v, delta=dz[1]-dz[0])
dwdy = mpcalc.first_derivative(w, delta=dy[1]-dy[0])
dudz = mpcalc.first_derivative(u, delta=dz[1]-dz[0])

def tilting_term(dwdy, dudz, dwdx, dvdz):
    return(dwdy*dudz + dwdx*dvdz)
tilt_term = tilting_term(dwdy,dudz,dwdy,dvdz)

**Soldenoidal term.**

**Local tendency.**

In [10]:
dvortdt = horiz_adv.values + vert_adv.values + stretch_term*units('s^2') + tilt_term*units('s^2') # + solenoid_term